In [5]:

from utils.spoon_api import *
from langchain_openai import OpenAI
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS


In [6]:
load_dotenv()
openai_key = os.getenv("openai_key")
os.environ['OPENAI_API_KEY'] = openai_key

In [24]:
llm = OpenAI(temperature=0.7, max_tokens=700)
loaders = UnstructuredURLLoader(urls=["https://www.wellplated.com/blueberry-banana-smoothie/",
                                      "https://pinchofyum.com/fluffiest-blueberry-pancakes",
                                      "https://www.recipetineats.com/blueberry-cheesecake/",
                                      "https://cakesbymk.com/recipe/simple-blueberry-cake/"])
data = loaders.load()

[Document(metadata={'source': 'https://www.wellplated.com/blueberry-banana-smoothie/'}, page_content="This post may contain affiliate links.\n\nA completely nutritious, cool, and creamy smoothie that also tastes like blueberry pie? Count me in! This healthy, easy Blueberry Smoothie made with yogurt truly delivers the best of all worlds.\n\nEmail Me the Recipe!\n\nEnter your email below to save this recipe to your free Well Plated recipe box.\n\nFrom time to time, we'll send you the best of Well Plated. Already registered? Log in here.\n\nWhy You’ll Love This Easy Blueberry Smoothie\n\nIt Truly Tastes DELISH. Like my Watermelon Smoothie and Strawberry Smoothie, this blueberry smoothie tastes more decadent than it is. It’s bursting with fresh blueberry taste, yes, but what sets this Greek yogurt blueberry banana smoothie apart is that its flavor is reminiscent of a slice of fresh blueberry pie with a creamy scoop of ice cream. Yum and YUM!\n\nIt’s Packed with Nutrients. This smoothie (an

In [36]:
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)
docs = text_spliter.split_documents(data)
len(docs)

74

In [37]:
embedder = OpenAIEmbeddings()
vector_index = FAISS.from_documents(docs, embedder)

In [38]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # or use "map_reduce" / "refine" if applicable
    retriever=vector_index.as_retriever(),
    return_source_documents=True
)
chain

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x73937a3e3b60>, async_client=<openai.resources.completions.AsyncCompletions object at 0x73937a51e420>, max_tokens=700, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmb

In [41]:
import langchain

query = "Ingredients for pancake with blueberry?"
langchain.debug = True

chain({"query": query})

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Ingredients for pancake with blueberry?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Ingredients for pancake with blueberry?",
  "context": "Ingredients\n\nScale\n\n3/4 cup milk\n\n2 tablespoons white vinegar\n\n1 cup flour\n\n2 tablespoons sugar\n\n1 teaspoon baking powder\n\n1/2 teaspoon baking soda\n\n1/2 teaspoon salt\n\n1 egg\n\n2 tablespoons melted butter\n\n1+ cup fresh blueberries\n\nmore butter for the pan\n\nInstructions\n\nMix the milk and vinegar and let it sit for a minute or two (you’re making “buttermilk” here).\n\nWhisk the dry ingredients together. Whisk the egg, milk, and melted butter into the dry ingredients until just combined.\n\nHeat a nonstick pan over medium heat. Melt a little smear of butter in the

{'query': 'Ingredients for pancake with blueberry?',
 'result': ' To make blueberry pancakes, you will need the following ingredients: 3/4 cup milk, 2 tablespoons white vinegar, 1 cup flour, 2 tablespoons sugar, 1 teaspoon baking powder, 1/2 teaspoon baking soda, 1/2 teaspoon salt, 1 egg, 2 tablespoons melted butter, 1+ cup fresh blueberries, more butter for the pan.',
 'source_documents': [Document(metadata={'source': 'https://pinchofyum.com/fluffiest-blueberry-pancakes'}, page_content='Ingredients\n\nScale\n\n3/4 cup milk\n\n2 tablespoons white vinegar\n\n1 cup flour\n\n2 tablespoons sugar\n\n1 teaspoon baking powder\n\n1/2 teaspoon baking soda\n\n1/2 teaspoon salt\n\n1 egg\n\n2 tablespoons melted butter\n\n1+ cup fresh blueberries\n\nmore butter for the pan\n\nInstructions\n\nMix the milk and vinegar and let it sit for a minute or two (you’re making “buttermilk” here).\n\nWhisk the dry ingredients together. Whisk the egg, milk, and melted butter into the dry ingredients until just c